# 데이터 로드

https://www.kaggle.com/datasets/sitaberete/python-datascience-handbook-dataset-md

In [1]:
'''
    from kaggle_secrets import UserSecretsClient
    import os

    os.environ["LANGCHAIN_TRACING_V2"] = 'true'
    os.environ["LANGCHAIN_API_KEY"] = UserSecrets
'''

'\n    from kaggle_secrets import UserSecretsClient\n    import os\n\n    os.environ["LANGCHAIN_TRACING_V2"] = \'true\'\n    os.environ["LANGCHAIN_API_KEY"] = UserSecrets\n'

In [2]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    '/root/rag_test/data',
    loader_cls = TextLoader,
    glob = '*.md',
    show_progress = True,
    exclude = [
        '05.15-Learning-More.md',
        '06.00-Figure-Code.md'
    ]
)

docs = loader.load()

100%|██████████| 63/63 [00:00<00:00, 19351.24it/s]


# 색인화(Indexing)

- Faiss

    1. 대규모 데이터셋을 다룰 때 (수백만 개 이상의 벡터)

    2. 매우 빠른 검색 속도가 필요한 경우
    
    3. 저수준 제어가 필요한 고도로 최적화된 시스템을 구축할 때

- Chroma

    1. Chroma는 LangChain과의 통합이 매우 간단하고 직관적입니다.

    2. Chroma는 단순한 벡터 검색을 넘어 문서 저장, 메타데이터 관리, 전체 텍스트 검색 등 더 많은 기능을 제공합니다.(RAG에 필요한 기능들 기본적으로 제공)

    3. 소규모부터 중간 규모의 프로젝트에 적합하며, 클라우드 확장성도 제공

## Massive Text Embedding

엄청나게 긴 글을 임베딩 하기에는 많은 문제가 있지만 무엇보다 모델의 토큰 수 제한이 가장 큰 걸림돌이다.

BERT만 생각해도 512토큰이 한계이고 최근 큰 모델도 2048토큰 정도이다.

하지만 이것보다 2048단어보다 더 긴 글을 임베딩 해야 할때는 어떻게 해야 할까?

[https://huggingface.co/spaces/mteb/leaderboard]

긴글을 임베딩 잘하는 모델들을 모아둔 leaderboard 이다. 

최근 모델에서는 NV-Embed-v2 모델이 1위 인데 우리가 사용하는 BAAI/bge-small-en 모델은 2위를 차지하고 있다.(해당 notebook이 만들어 졌을 때는 1위)

어떻게 해결했는지 각 모델을 공부해야 알 것 같다.

예전에 사용한 SBERT는 Sigment network(샴 네트워크) 구조를 사용해서 finetuning 함으로써 해결했었다.(? 사실상 검색에 더 적합한 임베딩을 만든 것이지 긴 글을 해결한 것은 아니다...)


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.retrievers import ParentDocumentRetriever

parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 100)

embedding_model = HuggingFaceBgeEmbeddings(model_name = 'BAAI/bge-small-en')

child_docs_store = Chroma(
    collection_name = "split_parents",
    embedding_function = embedding_model
)

parent_docs_store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore = child_docs_store,
    docstore = parent_docs_store,
    child_splitter = child_splitter,
    parent_splitter = parent_splitter,
    search_kwargs = {"k" : 1},
)

retriever.add_documents(docs)

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_1892005/2702342486.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  child_docs_store = Chroma(
